In [ ]:
import os
import json
import multiprocessing

import pandas as pd

from gurobi_models import KCMC_Result, GurobiModelWrapper

from tqdm.notebook import tqdm
from datetime import timedelta

In [ ]:
def parse_file(file):
    
    # Load the raw data from the file
    with open('/data/results/'+file, 'r') as fin:
        raw_data = json.load(fin)

    # Parse the PRESOLVE data
    ini_rows,  ini_cols,  ini_n0, \
    ini_cont,  ini_int,   ini_bin, \
    end_rows,  end_cols,  end_n0, \
    end_cont,  end_int,   end_bin, \
    prem_rows, prem_cols, pre_t, \
    gurobi_heuristic_objective_value = KCMC_Result.parse_presolve(raw_data['gurobi_logs'])

    assert (ini_rows-prem_rows) == end_rows, 'ROWS'
    assert (ini_cols-prem_cols) == end_cols, 'COLUMNS'

    # Parse some other fields
    pois, sensors, sinks = raw_data['key'].split(';')[1].split(' ')
    area_s, cov_r, com_r = raw_data['key'].split(';')[2].split(' ')
    gurobi_setup_time = sum([raw_data['time']['setup']['model']]
                      + list(raw_data['time']['setup']['constraints'].values())
                      + list(raw_data['time']['setup']['vars'].values()))

    # Parse the Gurobi solution to the KCMC problem
    X = KCMC_Result.parse_variable(raw_data['variables']['x'])
    solution = KCMC_Result.get_solution(X, int(sensors))

    # Emit a KCMC_Result object
    return KCMC_Result(
        source_file=file,

        # Key Values ----------------------
        pois=int(pois),
        sensors=int(sensors),
        sinks=int(sinks),

        area_side=int(area_s),
        coverage_radius=int(cov_r),
        communication_radius=int(com_r),

        random_seed=int(raw_data['random_seed']),

        k=int(raw_data['kcmc_k']),
        m=int(raw_data['kcmc_m']),

        heuristic_name=str(raw_data['preprocessing'].get('method', None)),
        y_binary=bool(raw_data['gurobi_y_binary']),
        gurobi_model_name=str(raw_data['gurobi_model']),

        # Main results --------------------
        heuristic_solution=str(raw_data['preprocessing'].get('solution', '1'*int(sensors))),
        # heuristic_objective_value
        # heuristic_solution_size
        # heuristic_solution_quality
        gurobi_optimal=bool(raw_data['status'] == 'OPTIMAL'),
        gurobi_heuristic_objective_value=gurobi_heuristic_objective_value,
        gurobi_objective_value=float(raw_data['objective_value']),
        gurobi_solution=solution,
        solution=solution,
        # solution_size
        # solution_quality

        # Time ----------------------------
        heuristic_time=float(raw_data['preprocessing'].get('runtime_us', 0)) / 1_000_000,
        gurobi_setup_time=float(gurobi_setup_time/1_000_000_000),
        gurobi_presolve_time=float(pre_t),
        gurobi_run_time=float(raw_data['gurobi_runtime']),
        # total_time: float

        # Other Solver results ------------
        gurobi_mip_gap=float(raw_data['json_solution']['SolutionInfo']['MIPGap']),
        gurobi_bound=float(raw_data['json_solution']['SolutionInfo']['ObjBound']),
        gurobi_bound_c=float(raw_data['json_solution']['SolutionInfo']['ObjBoundC']),
        gurobi_node_count=int(raw_data['json_solution']['SolutionInfo']['NodeCount']),
        gurobi_solutions_count=int(raw_data['json_solution']['SolutionInfo']['SolCount']),
        gurobi_simplex_iterations_count=int(raw_data['json_solution']['SolutionInfo']['IterCount']),
        gurobi_initial_rows_count=int(ini_rows),
        gurobi_initial_columns_count=int(ini_cols),
        gurobi_initial_non_zero_count=int(ini_n0),
        gurobi_initial_continuous_variables_count=int(ini_cont),
        gurobi_initial_integer_variables_count=int(ini_int),
        gurobi_initial_binary_variables_count=int(ini_bin),
        gurobi_presolve_removed_rows=int(prem_rows),
        gurobi_presolve_removed_columns=int(prem_cols),
        gurobi_rows_count=int(end_rows),
        gurobi_columns_count=int(end_cols),
        gurobi_non_zero_count=int(end_n0),
        gurobi_continuous_variables_count=int(end_cont),
        gurobi_integer_variables_count=int(end_int),
        gurobi_binary_variables_count=int(end_bin),

        # Other attributes ----------------
        time_limit=float(raw_data['time_limit']),
        gurobi_logs=str(raw_data['gurobi_logs']),
        gurobi_variable_x_size=len(X),
        gurobi_variable_y_size=KCMC_Result.get_variable_size(raw_data['variables']['y'])
    )


pre_df = pd.read_parquet('results.pq') if 'results.pq' in os.listdir() else None
pre_parsed_files = set(pre_df['source_file']) if pre_df else set()
dir_files = os.listdir('/data/results')
json_files = sorted([f for f in dir_files if f.endswith('.json') and f not in pre_parsed_files])

pool = multiprocessing.Pool()
parsed_results = list(
    tqdm(
        pool.imap_unordered(parse_file, json_files),
        total=len(json_files)
    )
)
pool.close()

df = pd.DataFrame([d.to_dict() for d in parsed_results])
df = df[sorted(df.columns)].copy()

len(dir_files), len(json_files), len(df), df.columns

In [ ]:
if pre_df is not None: df = pre_df.append(df).drop_duplicates().reset_index(drop=True)
df.to_parquet('results.pq')

## Check the Quantities of Processed Instances

In [ ]:
sdf = df[[
    'pois', 'sensors', 'sinks',
    # 'area_side', 'coverage_radius', 'communication_radius',
    'k', 'm',
    'heuristic_name', 'gurobi_model_name', 'y_binary',
    'random_seed'
]].copy()

sdf = sdf.groupby(list(sdf.columns[:-1])).nunique().sort_index().reset_index()

expected_total  = 6 * 6  # |{Instance sizes}| * |{(K,M) pairs}|
expected_total *= 10     # |{different seeds per group}|
expected_total *= 2 * 2  # |{different gurobi models}|
expected_total *= 1      # |{different pre-heuristics}|  # TODO

total_processed = int(sdf['random_seed'].sum())
print('\n\nDONE:',
      f'{total_processed}/{expected_total}',
      f'({round(100*(total_processed/expected_total), 3)}%)')

total_runtime = int(sum(df['total_time']))
total_expected_time = int((expected_total/total_processed)*total_runtime)
print('PROCESS TIME:', timedelta(seconds=total_runtime),
      f'\t({round(len(df)/(total_runtime/3600), 2)} tests/hour)',
      f'\t({round((total_runtime/60)/len(df), 2)} minutes/test)')

PARALLELISM = 6
ESTIMATED_AVERAGE_TEST_TIME = (total_runtime/len(df)) * 1.5
estimated_total_time = ESTIMATED_AVERAGE_TEST_TIME * expected_total
estimated_remaining_time = ESTIMATED_AVERAGE_TEST_TIME * (expected_total-total_processed)
print('ESTIMATED REMAINING WALL TIME:', timedelta(seconds=int(estimated_remaining_time/PARALLELISM)))

print('\n\nComplete:')
sdf[sdf['random_seed'] == 10]

In [ ]:
print('\n\nIncomplete:')
sdf[sdf['random_seed'] != 10]

In [ ]:
import plotly.express as px
fig = px.box(df, y="gurobi_run_time", x='gurobi_model_name')
fig.show()